# Что мы будем делать?
Посмотрим пример того, как с помощью Python и библиотек DS-стека делать развезодочный анализ данных (EDA).

<b>Разведочный анализ данных (Exploratory Data Analysis, EDA)</b> относится к критическому процессу выполнения первоначальных исследований данных с целью обнаружения закономерностей, выявления аномалий, проверки гипотез и предположений с помощью сводной статистики и графических представлений.

В качестве датасета для экспериментов возьмем табличные данные о сделках по продаже недвижимости https://www.kaggle.com/competitions/sberbank-russian-housing-market

## Раздел 1: импортируем библиотеки Python и загружаем табличные данные

[pandas](https://pandas.pydata.org/) - библиотека для анализа и манипуляций с данными

[seaborn](https://seaborn.pydata.org/index.html) - библиотека для визуализации статистических данных

[scipy](https://docs.scipy.org/doc/scipy/tutorial/general.html) - библиотека для статистического анализа

In [ ]:
#!pip install seaborn

In [ ]:
#Импортируем необходимые для работы библиотеки
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import os

pd.set_option('display.max_columns', 500)

In [ ]:
#Какие файлы в директории data?
os.listdir('data')

In [ ]:
#загружаем датафрейм в память из файла
dataframe = pd.read_csv(filepath_or_buffer='data/train.csv')

In [ ]:
#размеры таблицы с данными
dataframe.shape

In [ ]:
dataframe.info(memory_usage='deep')

In [ ]:
#первые 2 строчки датафрейма
dataframe.head(2)

In [ ]:
use_cols = ['id', 'timestamp', 'full_sq', 'floor', 'num_room', 'kitch_sq', 'park_km', 'kremlin_km', 'sub_area', 'price_doc']

dataframe = dataframe = pd.read_csv(filepath_or_buffer='data/train.csv', usecols=use_cols)

#размер таблицы сильно уменьшился
dataframe.info(memory_usage='deep')

In [ ]:
dataframe.shape

In [ ]:
dataframe.sample(2)

In [ ]:
#можно обращаться к конкретным столбцам таблицы через список 
dataframe[['timestamp', 'sub_area']].head(2)

## Раздел 2: есть ли рост средней стоимости квартиры и квадратного времени с течением времени?

Посмотрим динамику двух параметров: 
- динамику медианной цены квартиры по годам
- динамику средней цены квадратного метра по годам

Создадим в датафрейме новый вычисляемый столбец - <b>год</b> на основе <b>timestamp</b> сделки:

In [ ]:
dataframe['year'] = dataframe['timestamp'].apply(pd.to_datetime).dt.year

In [ ]:
dataframe[['timestamp', 'year']].sample(3)

Дополнительно создадим вычислимый столбец с ценой квадратного метра по формуле:
$$ meterprice= \frac{price\_doc}{full\_sq} $$

In [ ]:
dataframe['meter_price'] = dataframe['price_doc']/dataframe['full_sq']

In [ ]:
dataframe[['year','meter_price', 'full_sq', 'price_doc']].sample(3)

Группируем записи по году сделки, используя функцию groupby. 

Затем к группированным значениям применяем агрегирующие функции count, median, mean.

In [ ]:
dataframe.groupby('year').agg({'price_doc' : ['median', 'count'], 'meter_price' : ['mean']})

Почему за 2014 год средняя цена квадратного метра имеет значение NaN? Выясним:

In [ ]:
dataframe[(dataframe.year == 2014) & (dataframe.full_sq == 0)]

Причина в нулевой площади, нужно почистить некорректные данные. 

Всегда нужно оценивать, насколько бизнес-данные совпадают с логикой, и если что-то не сходится - следует исследовать причины.

In [ ]:
df_price_grouped = dataframe[dataframe.full_sq > 0]\
.groupby('year').agg({'price_doc' : ['median', 'count'], 'meter_price' : ['mean', 'median']})

df_price_grouped

Построим график средней цены квадратного метра в динамике 2011-2015 помесячно:

In [ ]:
dataframe['Year_month'] = dataframe['timestamp'].apply(pd.to_datetime).dt.strftime('%Y-%m')

df_monthly_price = dataframe.groupby('Year_month')[['meter_price']].mean().reset_index()

df_monthly_price.head(2)

In [ ]:
plt.figure(figsize=(20,8))
sns.set_theme(style="darkgrid")
plot = sns.lineplot(x='Year_month', y='meter_price', data=df_monthly_price)
plt.xticks(rotation=45)

plt.title('Цена квадратного метра в динамике лет')
plt.show()

## Раздел 3: визуализация распределений. 

Как еще можно посмотреть динамику и распределение цен, кроме как линейным графиком? Подойдет отличный графический инструмент для распределений - "ящик с усами" [(boxplot)](https://ru.wikipedia.org/wiki/%D0%AF%D1%89%D0%B8%D0%BA_%D1%81_%D1%83%D1%81%D0%B0%D0%BC%D0%B8).

![title](boxplot.png)


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=dataframe, x='year', y='meter_price', showfliers = False)

plt.title('Цена квадратного метра в динамике лет')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=dataframe[dataframe.num_room.isin([1,2,3])], 
            x='num_room', 
            y='meter_price', 
            showfliers = False, 
            hue='year')

plt.title('Цена квадратного метра в динамике лет')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=dataframe[dataframe.num_room.isin([1,2,3])], 
            x='year', 
            y='meter_price', 
            showfliers = True, 
            hue='num_room')

plt.title('Цена квадратного метра в динамике лет')
plt.show()

<b>Вывод:</b> Скачок средних цен в 2014 году связан с большим количеством аномалий (выбросов) в данных

## Раздел 4: а что со статистикой по районам?
Нам помогут:
- топ значения в категориях
- сводные таблицы [pivot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html)

In [ ]:
dataframe['sub_area'].value_counts()[:20]

In [ ]:
dataframe['sub_area'].value_counts(normalize=True)[:20]

In [ ]:
sub_list = list(dataframe['sub_area'].value_counts(normalize=True)[:10].index)

In [ ]:
dataframe[(dataframe.sub_area.isin(sub_list))]\
.groupby(['sub_area', 'year'])[['price_doc']].median()\
.reset_index().pivot_table(columns=['sub_area'], values='price_doc', index='year')

## Раздел 5: правда ли, что 1-комнатные квартиры в двух соседних районах статистически значимо различаются по площади?

Нам поможет метод проверки статистической значимости гипотез.

Возьмем 2 района (Poselenie Sosenskoe, Poselenie Moskovskij) и 1-шки в нем за 2013 год, будем проверять различия распределений пллощадей квартир.

In [ ]:
df = dataframe[(dataframe.year==2013) \
               & (dataframe.sub_area.isin(['Poselenie Moskovskij', 'Poselenie Sosenskoe'])) \
               & (dataframe.num_room == 1)]

In [ ]:
df.sub_area.value_counts()

In [ ]:
df.groupby('sub_area')[['full_sq']].agg(['mean'])

In [ ]:
_, p_value = scipy.stats.mannwhitneyu(x=df[df.sub_area == 'Poselenie Moskovskij'].full_sq.values,
                         y=df[df.sub_area == 'Poselenie Sosenskoe'].full_sq.values,
                         alternative='greater')

In [ ]:
alpha = 0.05 #уровень значимости критерия
print(round(p_value, 4))
if p_value < alpha:
    print("""Отклоняем гипотезу о раввенстве площадей, \nпринимаем альтернативу - в Московском поселении квартиры больше""")
else:
    print("""Нет оснований отвергнуть гипотезу о равенстве площадей""")